### 1. Imports, Configuration

In [2]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler 

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier

from scipy.stats import randint, loguniform


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### 2. Reading, splitting data

In [4]:
dfs = []
for f in Path("data").iterdir():
    df = pd.read_csv(f)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)    

In [5]:
df_playoff = pd.read_csv("playoff.csv")
df = df.merge(
    df_playoff,
    on=["team", "season"],
    how="left"
)

In [6]:
df.to_csv("raw_data.csv", index=False) # keeping raw data as a separate file, a fallback

In [7]:
df = df[df["situation"] == "all"] # only using combined metric(5v5 + 4v5 + 5v4...) for predictions
df.shape

(554, 110)

In [8]:
df_train, df_test = train_test_split(df, random_state=67)

### 3. EDA

In [9]:
df_train.head()

,team,season,name,team.1,position,situation,games_played,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,freezeFor,playStoppedFor,playContinuedInZoneFor,playContinuedOutsideZoneFor,savedShotsOnGoalFor,savedUnblockedShotAttemptsFor,penaltiesFor,penalityMinutesFor,faceOffsWonFor,hitsFor,takeawaysFor,giveawaysFor,lowDangerShotsFor,mediumDangerShotsFor,highDangerShotsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor,highDangerxGoalsFor,lowDangerGoalsFor,mediumDangerGoalsFor,highDangerGoalsFor,scoreAdjustedShotsAttemptsFor,unblockedShotAttemptsFor,scoreAdjustedUnblockedShotAttemptsFor,dZoneGiveawaysFor,xGoalsFromxReboundsOfShotsFor,xGoalsFromActualReboundsOfShotsFor,reboundxGoalsFor,totalShotCreditFor,scoreAdjustedTotalShotCreditFor,scoreFlurryAdjustedTotalShotCreditFor,xOnGoalAgainst,xGoalsAgainst,xReboundsAgainst,xFreezeAgainst,xPlayStoppedAgainst,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryAdjustedxGoalsAgainst,scoreVenueAdjustedxGoalsAgainst,flurryScoreVenueAdjustedxGoalsAgainst,shotsOnGoalAgainst,missedShotsAgainst,blockedShotAttemptsAgainst,shotAttemptsAgainst,goalsAgainst,reboundsAgainst,reboundGoalsAgainst,freezeAgainst,playStoppedAgainst,playContinuedInZoneAgainst,playContinuedOutsideZoneAgainst,savedShotsOnGoalAgainst,savedUnblockedShotAttemptsAgainst,penaltiesAgainst,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,penalitiesFor,penalitiesAgainst,playoff
1811,MTL,2019,MTL,MTL,Team Level,all,71,0.53,0.53,0.52,258596.0,2386.76,223.07,163.00,558.60,78.11,1309.56,969.79,213.98,224.14,215.04,2424.0,880.0,1148.0,4452.0,208.0,188.0,34.0,532.0,106.0,1053.0,1217.0,2216.0,3096.0,229.0,490.0,2140.0,1882.0,489.0,874.0,2434.0,626.0,244.0,64.85,77.88,80.34,66.0,67.0,75.0,4475.95,3304.0,3322.10,676.0,35.76,39.18,39.18,219.52,220.49,215.66,2142.02,201.00,150.35,501.51,71.42,1196.33,869.51,193.64,201.39,193.99,2210.0,782.0,928.0,3920.0,220.0,162.0,31.0,520.0,72.0,945.0,1073.0,1990.0,2772.0,205.0,442.0,2104.0,1693.0,531.0,911.0,2229.0,564.0,199.0,61.68,70.37,68.95,69.0,86.0,65.0,3920.91,2992.0,2993.71,686.0,33.29,31.16,31.21,202.96,203.28,198.78,NaN,NaN,0
2231,MIN,2022,MIN,MIN,Team Level,all,82,0.51,0.50,0.51,301108.0,2614.25,258.75,181.28,591.90,85.41,1428.64,1061.03,246.01,260.21,247.45,2535.0,1072.0,1187.0,4794.0,239.0,245.0,35.0,533.0,74.0,1159.0,1357.0,2296.0,3368.0,336.0,781.0,2181.0,1784.0,541.0,533.0,2572.0,775.0,260.0,73.51,94.96,90.27,60.0,98.0,81.0,4829.45,3607.0,3632.57,269.0,40.65,49.90,49.90,249.50,250.89,242.84,2532.86,248.36,173.68,585.17,84.28,1401.06,1008.46,235.35,247.30,234.40,2552.0,949.0,1318.0,4819.0,219.0,225.0,27.0,480.0,98.0,1128.0,1351.0,2333.0,3282.0,336.0,790.0,2393.0,1638.0,475.0,590.0,2535.0,720.0,246.0,74.37,86.14,87.86,69.0,84.0,66.0,4792.63,3501.0,3488.72,327.0,38.38,47.43,47.53,239.21,238.53,231.07,NaN,NaN,1
1046,OTT,2014,OTT,OTT,Team Level,all,82,0.52,0.51,0.50,301584.0,2523.50,225.45,168.13,582.83,81.27,1399.27,1056.04,217.68,225.92,218.15,2540.0,973.0,1300.0,4813.0,232.0,176.0,40.0,521.0,76.0,1134.0,1374.0,2308.0,3281.0,325.0,759.0,2421.0,2279.0,648.0,719.0,2691.0,606.0,216.0,73.86,72.94,78.64,69.0,76.0,87.0,4

In [10]:
df_train.shape

(415, 110)

In [11]:
df_train.head()

,team,season,name,team.1,position,situation,games_played,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,freezeFor,playStoppedFor,playContinuedInZoneFor,playContinuedOutsideZoneFor,savedShotsOnGoalFor,savedUnblockedShotAttemptsFor,penaltiesFor,penalityMinutesFor,faceOffsWonFor,hitsFor,takeawaysFor,giveawaysFor,lowDangerShotsFor,mediumDangerShotsFor,highDangerShotsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor,highDangerxGoalsFor,lowDangerGoalsFor,mediumDangerGoalsFor,highDangerGoalsFor,scoreAdjustedShotsAttemptsFor,unblockedShotAttemptsFor,scoreAdjustedUnblockedShotAttemptsFor,dZoneGiveawaysFor,xGoalsFromxReboundsOfShotsFor,xGoalsFromActualReboundsOfShotsFor,reboundxGoalsFor,totalShotCreditFor,scoreAdjustedTotalShotCreditFor,scoreFlurryAdjustedTotalShotCreditFor,xOnGoalAgainst,xGoalsAgainst,xReboundsAgainst,xFreezeAgainst,xPlayStoppedAgainst,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryAdjustedxGoalsAgainst,scoreVenueAdjustedxGoalsAgainst,flurryScoreVenueAdjustedxGoalsAgainst,shotsOnGoalAgainst,missedShotsAgainst,blockedShotAttemptsAgainst,shotAttemptsAgainst,goalsAgainst,reboundsAgainst,reboundGoalsAgainst,freezeAgainst,playStoppedAgainst,playContinuedInZoneAgainst,playContinuedOutsideZoneAgainst,savedShotsOnGoalAgainst,savedUnblockedShotAttemptsAgainst,penaltiesAgainst,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,penalitiesFor,penalitiesAgainst,playoff
1811,MTL,2019,MTL,MTL,Team Level,all,71,0.53,0.53,0.52,258596.0,2386.76,223.07,163.00,558.60,78.11,1309.56,969.79,213.98,224.14,215.04,2424.0,880.0,1148.0,4452.0,208.0,188.0,34.0,532.0,106.0,1053.0,1217.0,2216.0,3096.0,229.0,490.0,2140.0,1882.0,489.0,874.0,2434.0,626.0,244.0,64.85,77.88,80.34,66.0,67.0,75.0,4475.95,3304.0,3322.10,676.0,35.76,39.18,39.18,219.52,220.49,215.66,2142.02,201.00,150.35,501.51,71.42,1196.33,869.51,193.64,201.39,193.99,2210.0,782.0,928.0,3920.0,220.0,162.0,31.0,520.0,72.0,945.0,1073.0,1990.0,2772.0,205.0,442.0,2104.0,1693.0,531.0,911.0,2229.0,564.0,199.0,61.68,70.37,68.95,69.0,86.0,65.0,3920.91,2992.0,2993.71,686.0,33.29,31.16,31.21,202.96,203.28,198.78,NaN,NaN,0
2231,MIN,2022,MIN,MIN,Team Level,all,82,0.51,0.50,0.51,301108.0,2614.25,258.75,181.28,591.90,85.41,1428.64,1061.03,246.01,260.21,247.45,2535.0,1072.0,1187.0,4794.0,239.0,245.0,35.0,533.0,74.0,1159.0,1357.0,2296.0,3368.0,336.0,781.0,2181.0,1784.0,541.0,533.0,2572.0,775.0,260.0,73.51,94.96,90.27,60.0,98.0,81.0,4829.45,3607.0,3632.57,269.0,40.65,49.90,49.90,249.50,250.89,242.84,2532.86,248.36,173.68,585.17,84.28,1401.06,1008.46,235.35,247.30,234.40,2552.0,949.0,1318.0,4819.0,219.0,225.0,27.0,480.0,98.0,1128.0,1351.0,2333.0,3282.0,336.0,790.0,2393.0,1638.0,475.0,590.0,2535.0,720.0,246.0,74.37,86.14,87.86,69.0,84.0,66.0,4792.63,3501.0,3488.72,327.0,38.38,47.43,47.53,239.21,238.53,231.07,NaN,NaN,1
1046,OTT,2014,OTT,OTT,Team Level,all,82,0.52,0.51,0.50,301584.0,2523.50,225.45,168.13,582.83,81.27,1399.27,1056.04,217.68,225.92,218.15,2540.0,973.0,1300.0,4813.0,232.0,176.0,40.0,521.0,76.0,1134.0,1374.0,2308.0,3281.0,325.0,759.0,2421.0,2279.0,648.0,719.0,2691.0,606.0,216.0,73.86,72.94,78.64,69.0,76.0,87.0,4

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 415 entries, 1811 to 2266
Columns: 110 entries, team to playoff
dtypes: float64(102), int64(3), object(5)
memory usage: 359.9+ KB


In [13]:
df_train.describe()

,season,games_played,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,freezeFor,playStoppedFor,playContinuedInZoneFor,playContinuedOutsideZoneFor,savedShotsOnGoalFor,savedUnblockedShotAttemptsFor,penaltiesFor,penalityMinutesFor,faceOffsWonFor,hitsFor,takeawaysFor,giveawaysFor,lowDangerShotsFor,mediumDangerShotsFor,highDangerShotsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor,highDangerxGoalsFor,lowDangerGoalsFor,mediumDangerGoalsFor,highDangerGoalsFor,scoreAdjustedShotsAttemptsFor,unblockedShotAttemptsFor,scoreAdjustedUnblockedShotAttemptsFor,dZoneGiveawaysFor,xGoalsFromxReboundsOfShotsFor,xGoalsFromActualReboundsOfShotsFor,reboundxGoalsFor,totalShotCreditFor,scoreAdjustedTotalShotCreditFor,scoreFlurryAdjustedTotalShotCreditFor,xOnGoalAgainst,xGoalsAgainst,xReboundsAgainst,xFreezeAgainst,xPlayStoppedAgainst,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryAdjustedxGoalsAgainst,scoreVenueAdjustedxGoalsAgainst,flurryScoreVenueAdjustedxGoalsAgainst,shotsOnGoalAgainst,missedShotsAgainst,blockedShotAttemptsAgainst,shotAttemptsAgainst,goalsAgainst,reboundsAgainst,reboundGoalsAgainst,freezeAgainst,playStoppedAgainst,playContinuedInZoneAgainst,playContinuedOutsideZoneAgainst,savedShotsOnGoalAgainst,savedUnblockedShotAttemptsAgainst,penaltiesAgainst,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,penalitiesFor,penalitiesAgainst,playoff
count,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,393.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,393.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,22.000000,22.000000,415.000000
mean,2016.749398,75.233735,0.498843,0.499542,0.499373,274519.469880,2290.424747,214.467831,156.969036,526.062771,75.225084,1269.095855,942.051494,205.750964,214.995422,206.266675,2279.872289,910.163855,1088.568675,4278.604819,215.563855,179.026506,34.183133,505.209639,67.180723,1025.513253,1197.592771,2064.308434,2974.472289,291.977099,668.306024,2180.959036,1683.575904,509.053012,695.983133,2383.134940,598.768675,208.132530,68.752024,72.808434,72.906867,72.077108,73.975904,69.510843,4283.876193,3190.036145,3195.072867,384.669880,34.710048,39.113181,39.838289,208.932145,209.3687

In [14]:
df.select_dtypes(include=["object"]).columns.tolist()

['team', 'name', 'team.1', 'position', 'situation']

In [15]:
df.select_dtypes(include=["int64"]).columns.tolist()

['season', 'games_played', 'playoff']

In [16]:
with pd.option_context('display.max_rows', None):
    print(df_train.isna().any())

team                                         False
season                                       False
name                                         False
team.1                                       False
position                                     False
situation                                    False
games_played                                 False
xGoalsPercentage                             False
corsiPercentage                              False
fenwickPercentage                            False
iceTime                                      False
xOnGoalFor                                   False
xGoalsFor                                    False
xReboundsFor                                 False
xFreezeFor                                   False
xPlayStoppedFor                              False
xPlayContinuedInZoneFor                      False
xPlayContinuedOutsideZoneFor                 False
flurryAdjustedxGoalsFor                      False
scoreVenueAdjustedxGoalsFor    

In [17]:
with pd.option_context('display.max_rows', None):
    print(df_train.isnull().sum())

team                                           0
season                                         0
name                                           0
team.1                                         0
position                                       0
situation                                      0
games_played                                   0
xGoalsPercentage                               0
corsiPercentage                                0
fenwickPercentage                              0
iceTime                                        0
xOnGoalFor                                     0
xGoalsFor                                      0
xReboundsFor                                   0
xFreezeFor                                     0
xPlayStoppedFor                                0
xPlayContinuedInZoneFor                        0
xPlayContinuedOutsideZoneFor                   0
flurryAdjustedxGoalsFor                        0
scoreVenueAdjustedxGoalsFor                    0
flurryScoreVenueAdju

commented histogram code because very long output. feel free to uncomment, but weird stuff happens in notebook sometimes (at least in vscode)

In [18]:
# # this is without games_played normalization.
# cols_to_plot = df_train.select_dtypes(include=["float64"]).columns.tolist()
# fig, axes = plt.subplots(nrows=len(cols_to_plot), ncols=1, figsize=(10, 3*len(cols_to_plot)))

# for i, c in enumerate(cols_to_plot):
#     df_train.groupby("playoff")[c].plot.hist(
#         bins=50,
#         alpha=0.5, 
#         legend=True, 
#         density=True, 
#         ax=axes[i]
#     )
    
#     axes[i].set_title(f"Histogram of {c}")
#     axes[i].set_xlabel(c)

# plt.tight_layout()
# plt.show()


In [19]:
df_train[["xFreezeFor", "xPlayStoppedFor", "xPlayContinuedInZoneFor", "playoff"]].corr() 

,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,playoff
xFreezeFor,1.000000,0.994001,0.976669,0.112922
xPlayStoppedFor,0.994001,1.000000,0.986277,0.125181
xPlayContinuedInZoneFor,0.976669,0.986277,1.000000,0.133454
playoff,0.112922,0.125181,0.133454,1.000000


lets just include one of these...i'll choose xPlayContinuedInZoneFor because highest correlation with playoff (albeit minimally, should not read too much into it). as an extension i'll also use the other xPlayContinued(In/Outside)Zone(For/Against)
<br>
<br>

In [20]:
df_train[["xGoalsPercentage", "flurryScoreVenueAdjustedxGoalsFor", "playoff"]].corr()

,xGoalsPercentage,flurryScoreVenueAdjustedxGoalsFor,playoff
xGoalsPercentage,1.000000,0.343666,0.610125
flurryScoreVenueAdjustedxGoalsFor,0.343666,1.000000,0.197803
playoff,0.610125,0.197803,1.000000


In [21]:
df_train[["corsiPercentage", "fenwickPercentage", "xOnGoalFor", "playoff"]].corr()

,corsiPercentage,fenwickPercentage,xOnGoalFor,playoff
corsiPercentage,1.000000,0.964455,0.333351,0.434631
fenwickPercentage,0.964455,1.000000,0.318613,0.476752
xOnGoalFor,0.333351,0.318613,1.000000,0.130911
playoff,0.434631,0.476752,0.130911,1.000000


hmm... just fenwick or both? my gut is saying try out with both first so i'll do that. i wanna learn more systematic/data driven approaches to make these decisions rather than just my gut...
<br>
<br>

In [22]:
df_train[df_train["season"] == 2025][["xGoalsPercentage", 
                                      "xGoalsFor", 
                                      "xOnGoalFor", 
                                      "xPlayContinuedInZoneFor",
                                      "xPlayContinuedOutsideZoneFor",
                                      "xPlayContinuedInZoneAgainst",
                                      "xPlayContinuedOutsideZoneAgainst",
                                      "flurryScoreVenueAdjustedxGoalsFor",
                                      "lowDangerxGoalsFor",
                                      "mediumDangerxGoalsFor"]].head()

,xGoalsPercentage,xGoalsFor,xOnGoalFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryScoreVenueAdjustedxGoalsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor
2626,0.51,119.59,1271.80,686.13,496.12,637.67,472.58,112.91,36.78,37.11
2621,0.51,132.41,1310.37,702.01,511.82,700.09,500.51,126.21,40.74,41.35
2631,0.51,129.45,1357.50,733.88,522.22,644.06,462.66,124.77,40.97,43.64
2611,0.48,128.02,1312.02,710.66,504.20,706.48,502.84,121.41,39.56,43.13
2616,0.46,123.33,1324.86,727.08,512.48,758.93,542.46,119.39,37.16,42.03


In [23]:
df_train[df_train["season"] != 2025][["xGoalsPercentage", 
                                      "xGoalsFor", 
                                      "xOnGoalFor", 
                                      "xPlayContinuedInZoneFor",
                                      "xPlayContinuedOutsideZoneFor",
                                      "xPlayContinuedInZoneAgainst",
                                      "xPlayContinuedOutsideZoneAgainst",
                                      "flurryScoreVenueAdjustedxGoalsFor",
                                      "lowDangerxGoalsFor",
                                      "mediumDangerxGoalsFor"]].head()

,xGoalsPercentage,xGoalsFor,xOnGoalFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryScoreVenueAdjustedxGoalsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor
1811,0.53,223.07,2386.76,1309.56,969.79,1196.33,869.51,215.04,64.85,77.88
2231,0.51,258.75,2614.25,1428.64,1061.03,1401.06,1008.46,247.45,73.51,94.96
1046,0.52,225.45,2523.50,1399.27,1056.04,1436.51,1077.43,218.15,73.86,72.94
1146,0.51,254.52,2658.51,1486.16,1075.64,1405.96,1040.10,242.32,77.98,89.48
1076,0.48,222.66,2560.21,1418.25,1057.18,1406.15,1055.73,215.47,75.57,79.84


mhm all but goals percentage (and ofc corsi & fenwick) need adjusting (i will NOT be using xGoalsFor as xGoalsPercentage is derived from it)
<br>
<br>

### 4. Feature Engineering

we want features that can be used regardless of games played. only 3 in the current dataset fit this description ("xGoalsPercentage", "corsiPercentage", "fenwickPercentage"). we'll create some more ratios/percentages using relevant features and the EDA done above

-> these initial relevant features were identified by using histograms and corr(). some decisions were also made with my (limited) domain knowledge/gut instinct

features i chose in eda above:
- corsiPercentage
- fenwickPercentage
- xGoalsPercentage
- xOnGoalFor
- xPlayContinuedInZoneFor
- xPlayContinuedOutsideZoneFor
- xPlayContinuedInZoneAgainst
- xPlayContinuedOutsideZoneAgainst
- flurryScoreVenueAdjustedxGoalsFor
- lowDangerShotsFor
- lowDangerxGoalsFor
- lowDangerGoalsFor
- lowDangerShotsAgainst
- lowDangerxGoalsAgainst
- lowDangerGoalsAgainst

In [24]:
def engineer_features(df):
    df = df.copy()
    df["xOnGoalFor_PerGame"] = df["xOnGoalFor"] / df["games_played"]
    df["xPlayContinuedInZoneFor_PerGame"] = df["xPlayContinuedInZoneFor"] / df["games_played"]
    df["xPlayContinuedOutsideZoneFor_PerGame"] = df["xPlayContinuedOutsideZoneFor"] / df["games_played"]
    df["xPlayContinuedInZoneAgainst_PerGame"] = df["xPlayContinuedInZoneAgainst"] / df["games_played"]
    df["xPlayContinuedOutsideZoneAgainst_PerGame"] = df["xPlayContinuedOutsideZoneAgainst"] / df["games_played"]
    df["flurryScoreVenueAdjustedxGoalsFor_PerGame"] = df["flurryScoreVenueAdjustedxGoalsFor"] / df["games_played"]
    df["lowDangerShotsFor_PerGame"] = df["lowDangerShotsFor"] / df["games_played"]
    df["lowDangerxGoalsFor_PerGame"] = df["lowDangerxGoalsFor"] / df["games_played"]
    df["lowDangerGoalsFor_PerGame"] = df["lowDangerGoalsFor"] / df["games_played"]
    df["lowDangerShotsAgainst_PerGame"] = df["lowDangerShotsAgainst"] / df["games_played"]
    df["lowDangerxGoalsAgainst_PerGame"] = df["lowDangerxGoalsAgainst"] / df["games_played"]
    df["lowDangerGoalsAgainst_PerGame"] = df["lowDangerGoalsAgainst"] / df["games_played"]
    
    df = df[["corsiPercentage",
             "fenwickPercentage",
             "xGoalsPercentage",
             "xOnGoalFor_PerGame",
             "xPlayContinuedInZoneFor_PerGame",
             "xPlayContinuedOutsideZoneFor_PerGame",
             "xPlayContinuedInZoneAgainst_PerGame",
             "xPlayContinuedOutsideZoneAgainst_PerGame",
             "flurryScoreVenueAdjustedxGoalsFor_PerGame",
             "lowDangerShotsFor_PerGame",
             "lowDangerxGoalsFor_PerGame",
             "lowDangerGoalsFor_PerGame",
             "lowDangerShotsAgainst_PerGame",
             "lowDangerxGoalsAgainst_PerGame",
             "lowDangerGoalsAgainst_PerGame",
             "playoff"]]
    return df

In [25]:
df_train = engineer_features(df_train)
df_test = engineer_features(df_test)

In [26]:
print(df_train.shape)
print(df_test.shape)

(415, 16)
(139, 16)


In [27]:
df_train.describe()

,corsiPercentage,fenwickPercentage,xGoalsPercentage,xOnGoalFor_PerGame,xPlayContinuedInZoneFor_PerGame,xPlayContinuedOutsideZoneFor_PerGame,xPlayContinuedInZoneAgainst_PerGame,xPlayContinuedOutsideZoneAgainst_PerGame,flurryScoreVenueAdjustedxGoalsFor_PerGame,lowDangerShotsFor_PerGame,lowDangerxGoalsFor_PerGame,lowDangerGoalsFor_PerGame,lowDangerShotsAgainst_PerGame,lowDangerxGoalsAgainst_PerGame,lowDangerGoalsAgainst_PerGame,playoff
count,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000
mean,0.499542,0.499373,0.498843,30.430503,16.850833,12.491430,16.888999,12.512154,2.740694,31.640530,0.913229,0.960773,31.673920,0.915814,0.972173,0.508434
std,0.028053,0.027959,0.033015,2.166567,1.165354,0.885424,1.145318,0.832302,0.287476,2.304952,0.070897,0.166760,2.180172,0.071630,0.182931,0.500532
min,0.380000,0.390000,0.400000,23.309512,12.859268,9.654878,13.570732,10.158902,2.007805,23.548780,0.636341,0.463415,25.060976,0.712073,0.573171,0.000000
25%,0.480000,0.480000,0.480000,28.804573,16.040188,11.932425,16.077012,11.919390,2.548171,30.036585,0.867853,0.841463,30.255880,0.864512,0.841463,0.000000
50%,0.500000,0.500000,0.500000,30.376463,16.878393,12.460854,16.924024,12.494756,2.712899,31.597561,0.912561,0.951220,31.560976,0.914167,0.951220,1.000000
75%,0.520000,0.520000,0.520000,31.819024,17.564512,13.096890,17.580854,13.061762,2.913171,33.060976,0.954878,1.073171,33.071797,0.963232,1.085366,1.000000
max,0.600000,0.590000,0.590000,37.434146,20.428171,14.896220,21.376463,15.201220,3.719512,38.073171,1.166829,1.682927,37.865854,1.145122,1.585366,1.000000


std now reasonable numbers, now features can be used to analyze regardless of games played! 
(for an extra check, i also ran a session that kept season and compared 2025 and non 2025 numbers, the values were the same range)

### 5. Preprocessing (and a DummyClassifier first)

In [28]:
X_train = df_train.drop(columns=["playoff"])
y_train = df_train.loc[:, "playoff"]

X_test = df_test.drop(columns=["playoff"])
y_test = df_test.loc[:, "playoff"]

In [29]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(415, 15)
(415,)
(139, 15)
(139,)


In [30]:
dummy = DummyClassifier()
dummy_df = pd.DataFrame(cross_validate(dummy, X_train, y_train, return_train_score=True))
dummy_df

,fit_time,score_time,test_score,train_score
0,0.001344,0.001198,0.506024,0.509036
1,0.002041,0.001654,0.506024,0.509036
2,0.001658,0.001421,0.506024,0.509036
3,0.001994,0.001392,0.506024,0.509036
4,0.002788,0.001835,0.518072,0.506024


In [31]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 415 entries, 1811 to 2266
Data columns (total 15 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   corsiPercentage                            415 non-null    float64
 1   fenwickPercentage                          415 non-null    float64
 2   xGoalsPercentage                           415 non-null    float64
 3   xOnGoalFor_PerGame                         415 non-null    float64
 4   xPlayContinuedInZoneFor_PerGame            415 non-null    float64
 5   xPlayContinuedOutsideZoneFor_PerGame       415 non-null    float64
 6   xPlayContinuedInZoneAgainst_PerGame        415 non-null    float64
 7   xPlayContinuedOutsideZoneAgainst_PerGame   415 non-null    float64
 8   flurryScoreVenueAdjustedxGoalsFor_PerGame  415 non-null    float64
 9   lowDangerShotsFor_PerGame                  415 non-null    float64
 10  lowDangerxGoalsFor_PerGame 

In [32]:
X_train.isna().any().any().item()

False

all numeric, no missing values, all i need is StandardScaler(). no column transformer needed.

### 6. Training

In [33]:
classifiers = {
    "lr": LogisticRegression(max_iter=1000),
    "rbf_svm": SVC(),
    "random_forest": RandomForestClassifier(),
    "xgboost": XGBClassifier(),
    "lightgbm": LGBMClassifier(),
    "catboost": CatBoostClassifier(verbose=0)
}

In [34]:
mean_results = {}
std_results = {}

for name, classifier in classifiers.items():
    print(name)
    if name in ["lr", "rbf_svm"]:
        pipe = make_pipeline(StandardScaler(), classifier)
    else:
        pipe = make_pipeline(classifier)
    result = pd.DataFrame(cross_validate(pipe, X_train, y_train, return_train_score=True))
    
    mean_results[name] = result.mean()
    std_results[name] = result.std()

lr
rbf_svm
random_forest
xgboost
lightgbm
[LightGBM] [Info] Number of positive: 169, number of negative: 163
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000113 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1283
[LightGBM] [Info] Number of data points in the train set: 332, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.509036 -> initscore=0.036149
[LightGBM] [Info] Start training from score 0.036149
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

c:\Users\ryanh\code\tests\nhl-test-2\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:927: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\ryanh\code\tests\nhl-test-2\.venv\Lib\site-packages\sklearn\utils\_tags.py", line 275, in get_tags
    tags = estimator.__sklearn_tags__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'CatBoostClassifier' object has no attribute '__sklearn_tags__'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\ryanh\code\tests\nhl-test-2\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 916, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\ryanh\code\tests\nhl-test-2\.venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 485, in __call__
    return estimator.score(*args, **kwargs)
     

In [35]:
results = pd.DataFrame(mean_results).T
std_results = pd.DataFrame(std_results).T

In [ ]:
results = results.assign(test_score_std=std_results["test_score"])
results.sort_values(by=["test_score"], ascending=False)

,fit_time,score_time,test_score,train_score,test_score_std
lr,0.027531,0.004848,0.853012,0.859639,0.037524
rbf_svm,0.017365,0.007700,0.831325,0.871687,0.038100
xgboost,0.091124,0.004171,0.787952,1.000000,0.055605
lightgbm,0.408791,0.002357,0.780723,1.000000,0.033432
random_forest,0.259287,0.014179,0.771084,1.000000,0.037135
catboost,2.635915,0.002625,NaN,NaN,NaN


### Feature Importances

In [37]:
lr = make_pipeline(StandardScaler(), LogisticRegression())
lr.fit(X_train, y_train)
lr_coefs = pd.DataFrame(
    data=lr.named_steps["logisticregression"].coef_[0], index=X_train.columns, columns=["Coefficient"]
)
lr_coefs

,Coefficient
corsiPercentage,-1.039828
fenwickPercentage,0.415382
xGoalsPercentage,2.238536
xOnGoalFor_PerGame,-0.892088
xPlayContinuedInZoneFor_PerGame,-0.391934
xPlayContinuedOutsideZoneFor_PerGame,0.927625
xPlayContinuedInZoneAgainst_PerGame,0.456297
xPlayContinuedOutsideZoneAgainst_PerGame,-0.955761
flurryScoreVenueAdjustedxGoalsFor_PerGame,0.597036
lowDangerShotsFor_PerGame,-0.448102


### 7. Stacking model with randomized constituents

In [38]:
gammas = 10**np.random.uniform(-3, 3, 50)
cs = 10**np.random.uniform(-3, 3, 50)

models = [LogisticRegression(C=c, max_iter=1000) for c in np.logspace(-3, 3, 50)] + [SVC(gamma=gamma, C=c) for gamma, c in zip(gammas, cs)] 

In [39]:
named_models = [(f"model_{i}", model) for i, model in enumerate(models)]
stacking_model = StackingClassifier(named_models)
stacking_model.fit(X_train, y_train)

c:\Users\ryanh\code\tests\nhl-test-2\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\ryanh\code\tests\nhl-test-2\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/m

,"estimators estimators: list of (str, estimator)Base estimators which will be stacked together. Each element of thelist is defined as a tuple of string (i.e. name) and an estimatorinstance. An estimator can be set to 'drop' using `set_params`.The type of estimator is generally expected to be a classifier.However, one can pass a regressor for some use case (e.g. ordinalregression).","[('model_0', ...), ('model_1', ...), ...]"
,"final_estimator final_estimator: estimator, default=NoneA classifier which will be used to combine the base estimators.The default classifier is a:class:`~sklearn.linear_model.LogisticRegression`.",None
,"cv cv: int, cross-validation generator, iterable, or ""prefit"", default=NoneDetermines the cross-validation splitting strategy used in`cross_val_predict` to train `final_estimator`. Possible inputs forcv are:* None, to use the default 5-fold cross validation,* integer, to specify the number of folds in a (Stratified) KFold,* An object to be used as a cross-validation generator,* An iterable yielding train, test splits,* `""prefit""`, to assume the `estimators` are prefit. In this case, the estimators will not be refitted.For integer/None inputs, if the estimator is a classifier and y iseither binary or multiclass,:class:`~sklearn.model_selection.StratifiedKFold` is used.In all other cases, :class:`~sklearn.model_selection.KFold` is used.These splitters are instantiated with `shuffle=False` so the splitswill be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here.If ""prefit"" is passed, it is assumed that all `estimators` havebeen fitted already. The `final_estimator_` is trained on the `estimators`predictions on the full training set and are **not** cross validatedpredictions. Please note that if the models have been trained on the samedata to train the stacking model, there is a very high risk of overfitting... versionadded:: 1.1 The 'prefit' option was added in 1.1.. note:: A larger number of split will provide no benefits if the number of training samples is large enough. Indeed, the training time will increase. ``cv`` is not used for model evaluation but for prediction.",None
,"stack_method stack_method: {'auto', 'predict_proba', 'decision_function', 'predict'}, default='auto'Methods called for each base estimator. It can be:* if 'auto', it will try to invoke, for each estimator, `'predict_proba'`, `'decision_function'` or `'predict'` in that order.* otherwise, one of `'predict_proba'`, `'decision_function'` or `'predict'`. If the method is not implemented by the estimator, it will raise an error.",'auto'
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for `fit` of all `estimators`.`None` means 1 unless in a `joblib.parallel_backend` context. -1 meansusing all processors. See :term:`Glossary ` for more details.",None
,"passthrough passthrough: bool, default=FalseWhen False, only the predictions of estimators will be used astraining data for `final_estimator`. When True, the`final_estimator` is trained on the predictions as well as theoriginal training data.",False
,"verbose verbose: int, default=0Verbosity level.",0
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of r

In [40]:
named_models[0]

('model_0', LogisticRegression(C=np.float64(0.001), max_iter=1000))

In [41]:
pd.DataFrame(
    data=stacking_model.final_estimator_.coef_.flatten(),
    columns=["Coefficient"]
).sort_values("Coefficient", ascending=False)

,Coefficient
96,1.005025
48,0.713193
49,0.574194
47,0.552521
46,0.512125
...,...
51,-0.253970
89,-0.274311
88,-0.501365
54,-0.509266


In [42]:
stack_pipe = make_pipeline(StandardScaler(), stacking_model)
result = pd.DataFrame(cross_validate(stack_pipe, X_train, y_train, return_train_score=True))

results = pd.DataFrame(result.mean()).T
std_results = pd.DataFrame(result.std()).T
results = results.assign(test_score_std=std_results["test_score"])
results.sort_values(by=["test_score"], ascending=False)

,fit_time,score_time,test_score,train_score,test_score_std
0,3.065115,0.087229,0.843373,0.86506,0.045878


In [45]:
final_classifiers = {
    "lr": LogisticRegression(max_iter=1000),
    "rbf_svm": SVC(),
    "stacking": stacking_model}

In [47]:
mean_results = {}
std_results = {}

for name, classifier in final_classifiers.items():
    print(name)
    pipe = make_pipeline(StandardScaler(), classifier)
    result = pd.DataFrame(cross_validate(pipe, X_train, y_train, return_train_score=True))
    
    mean_results[name] = result.mean()
    std_results[name] = result.std()

lr
rbf_svm
stacking


In [48]:
results = pd.DataFrame(mean_results).T
std_results = pd.DataFrame(std_results).T
results = results.assign(test_score_std=std_results["test_score"])
results.sort_values(by=["test_score"], ascending=False)

,fit_time,score_time,test_score,train_score,test_score_std
lr,0.017517,0.003348,0.853012,0.859639,0.037524
stacking,2.716149,0.079312,0.843373,0.865060,0.045878
rbf_svm,0.014534,0.006191,0.831325,0.871687,0.038100
